In [1]:
import os, sys, json
sys.path.append("/workspace/AutoCog")
from autocog.language.sta.compile import compile
from autocog.utility.pynb import wrap_graphviz, display

In [2]:
program = compile("""
define default_number_of_choices=4;
argument N=$default_number_of_choices;
prompt mmlu_main {
    is {
        topic is text;
        question is text;
        choices[$N] is {
            value is text;
            correct is bool;
        }
        answer is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return {
        from .answer;
    }
    annotate {
        .topic           as "the general category from which the question was taken";
        .question        as "the question that you have to answer";
        .choices         as "you judge whether each choice is correct or not";
        .choices.value   as "the value of the choice";
        .choices.correct as "you decide whether this choice is correct or not";
        .answer          as "you repeat the value of the choice that best answer the question";
    }
}
""")
wrap_graphviz(program.toGraphViz())

NotImplementedError: 